In [2]:
from transformers import AutoModelForCausalLM, AutoProcessor, AutoConfig
import torch
from PIL import Image
import requests

In [10]:
from io import BytesIO


# Device setup
device = "cuda"

# Load Florence-2 model and processor
model_name = "gokaygokay/Florence-2-Flux-Large"
model = AutoModelForCausalLM.from_pretrained("gokaygokay/Florence-2-Flux-Large", trust_remote_code=True).to(device).eval()
processor = AutoProcessor.from_pretrained(model_name, trust_remote_code=True)

# Function to perform OCR using Florence-2
def run_ocr(image):
    task_prompt = "<OCR>"

    # Ensure the image is in RGB mode
    if image.mode != "RGB":
        image = image.convert("RGB")

    inputs = processor(text=task_prompt, images=image, return_tensors="pt").to(device)
    generated_ids = model.generate(
        input_ids=inputs["input_ids"],
        pixel_values=inputs["pixel_values"],
        max_new_tokens=512,  # OCR output is usually shorter
        num_beams=3,
        repetition_penalty=1.10,
    )
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]
    parsed_answer = processor.post_process_generation(generated_text, task=task_prompt, image_size=(image.width, image.height))
    return parsed_answer["<OCR>"]

# Load image
image_url = "https://raw.githubusercontent.com/gokul-1998/handwriting_recognition/main/test_images/test3.png"
response = requests.get(image_url)
image = Image.open(BytesIO(response.content))


# Perform OCR
ocr_text = run_ocr(image)
print(ocr_text)


Dear User,Handwrytten uses robotichandwriting machines that usean actual pen to write yourmessage. The results are verticallyindistinguishable from actualhandworthing.Try it today!The Robot
